In [1]:
import pandas as pd
import numpy as np
from random import randint
from random import seed

from math import radians, sin, cos, tan, asin, sqrt
from timeit import default_timer as timer

In [2]:
def haversine_new(lat1, lon1):
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    #reshape for broadcasting
    lat1 = lat1.reshape(len(lat1),1)
    lon1 = lon1.reshape(len(lat1),1)
    lat2 = lat1.reshape(1,len(lat1))
    lon2 = lon1.reshape(1,len(lat1))
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    d = np.sin(dlat * 0.5)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon * 0.5)**2
    c = 2.0 * 6371.0
    return c * np.arcsin(np.sqrt(d))

In [3]:
def weighted_trip_length_new(this_trip,hvs_matrix):
    #setup variable
    index = this_trip.index
    sleigh_weight = 10
    prev_dest = 0
    wrw = 0.0
    prev_weight = np.sum(this_trip.Weight) + sleigh_weight
    
    for i in index:
        next_dest = i
        wrw += hvs_matrix[prev_dest][next_dest] * prev_weight  #distance * weights#   
        prev_dest = next_dest
        prev_weight -= this_trip.loc[next_dest,'Weight']
    return wrw

In [4]:
def weighted_reindeer_weariness_new(all_trips, weight_limit = 1000):
    #check total weight for each trip
    if any(all_trips.groupby('TripId').Weight.sum() > weight_limit):
        raise Exception("One of the sleighs over weight limit!")
        
    lat1 = np.array(all_trips['Latitude'].tolist())
    lon1 = np.array(all_trips['Longitude'].tolist())
    hvs_matrix = haversine_new(lat1,lon1)    
    #calculate the wrw for each unique trip
    wrw = 0.0
    uniq_trips = all_trips.TripId.unique()
    for t in uniq_trips:
        this_trip = all_trips[all_trips.TripId==t]
        wrw += weighted_trip_length_new(this_trip,hvs_matrix)
    return wrw

In [7]:
def random_search(N):
    # 1000 * N solutions = max 
    N_eval = 1 * N
    df = pd.read_csv('gifts.csv')
    # Initial conditions 
    loop_1 = 0 #
    loop_2 = 0 
    seed_count = 0 
    best_wrw_list = []
    # Final result is the optimal WRW obtained. 
    RS_stat = pd.DataFrame(columns = ['Max', 'Min', 'Mean', 'Std']) 
    start_time_1 = timer()
  
    # loop 1 
    for i in range(30):
        seed = randint(1, 10000)
        best_wrw = None
        loop_1 += 1
        seed_count += 1
        RS_stat = pd.DataFrame(columns = ['Max', 'Min', 'Mean', 'Std']) 
    # Loop2  * N times
        for j in range(N_eval):
            start_time_2 = timer() # calc time of inner loop
            loop_2 += 1
            
            # Extract 10 random sample from gifts.csv
            df_10 = df.sample(n = 10, random_state = seed) #seed changes every loop, 30 times, 30 seeds
            # Converts GiftId column into list
            giftid = df_10['GiftId'].tolist()
            # Create 10 TripId in a vector
            tripid = np.array(range(11)) # 
            # Permutate the 10 TripId
            tripid_permutate = np.random.randint(10, size = 10) # size = shape size 
            # Match each GiftId with permutated TripId and put in dictionary
            giftid_tripid = dict(zip(giftid, tripid_permutate)) # key = gift, value = trip_id 
            # print(giftid_tripid)
            # Convert dict to dataframe
            giftid_tripid_df = pd.DataFrame(list(giftid_tripid.items()), 
                                            columns = ['GiftId', 'TripId'])
            #print(giftid_tripid_df)
            # Merge dataframe (GiftId & TripId) with 10 random sampled gifts.csv (df_10)
            all_trips = giftid_tripid_df.merge(df_10, on = 'GiftId')
            print(all_trips)
            # Calculate wrw of that dataset
            new_wrw = weighted_reindeer_weariness_new(all_trips)
            #print(wrw)
            if best_wrw is None:
               best_wrw = new_wrw
            # Replace result with new wrw only if the new wrw is larger than the initial or current result
            elif new_wrw > best_wrw:
                 best_wrw = new_wrw

            best_wrw_list.append(best_wrw)
            best_wrw_array = np.asarray(best_wrw_list)
            MEAN = np.mean(best_wrw_array)
            MAX = np.max(best_wrw_array)
            MIN = np.min(best_wrw_array)
            STD = np.std(best_wrw_array)
            
            end_time_2 = timer()
            # Within loop (tables)
            #print("WRW = {:.0f}  (Time: {:.2f} seconds)".format(wrw, end_time_2 - start_time_2))
        
        end_time_1 = timer()

        print('Seed', seed_count, seed, 'WRW:', best_wrw)

    print('Total runs:',loop_1)
    print('\n')
    print('Total evaluations:',loop_2)
    print('Total time: {:.2f} seconds)'.format(end_time_1 - start_time_1))

    print('Mean WRW:',np.mean(best_wrw_array))
    print('Max WRW:',np.max(best_wrw_array))
    print('Min WRW:', np.min(best_wrw_array))
    print('Standard deviation', np.std(best_wrw_array))
    # Print total time spend to get entire result
    print('Time: {:.2f} seconds)'.format(end_time_1 - start_time_1))
    RS_stat = RS_stat.append({'Max': MAX, 'Min': MIN, 'Mean': MEAN, 'Std':STD},
                            ignore_index = True)


In [8]:
# Run random_search with 10 sample data  
random_search(10)

   GiftId  TripId   Latitude   Longitude     Weight
0   21915       1  23.379164   80.776322   1.000000
1   49108       2  69.941627  -34.715030   2.645473
2   27723       6   4.751999    6.018371   1.000000
3   53466       2 -79.303203 -113.787137  33.505419
4   63932       7  24.113137   67.905691  34.896201
5   38091       5   8.513220    5.234561  42.371074
6   80805       0  -0.653591   33.386683   1.000000
7   99777       1  51.786099 -103.591722   8.180307
8   10056       5 -69.769628   14.209608   7.642927
9   10557       6  19.331321  109.068036  50.000000
   GiftId  TripId   Latitude   Longitude     Weight
0   21915       0  23.379164   80.776322   1.000000
1   49108       8  69.941627  -34.715030   2.645473
2   27723       8   4.751999    6.018371   1.000000
3   53466       2 -79.303203 -113.787137  33.505419
4   63932       6  24.113137   67.905691  34.896201
5   38091       7   8.513220    5.234561  42.371074
6   80805       6  -0.653591   33.386683   1.000000
7   99777   

   GiftId  TripId   Latitude   Longitude     Weight
0   57206       0  28.934005  112.925264   1.000000
1   55792       9 -16.404233  -62.097381  23.487230
2   50620       9  16.933804   95.222256  15.050914
3   73919       7 -76.396243  157.351948  39.734369
4   44664       0  49.559341   86.044948   1.000000
5   11342       2  36.063459   71.383547  29.348293
6   59135       7   7.960739   -6.260968   4.092837
7   38911       2 -29.654836  -54.852475   1.000000
8   69997       0  -2.151272   12.109744  18.834981
9   19366       6  59.288416   89.978393  30.876355
   GiftId  TripId   Latitude   Longitude     Weight
0   57206       1  28.934005  112.925264   1.000000
1   55792       0 -16.404233  -62.097381  23.487230
2   50620       9  16.933804   95.222256  15.050914
3   73919       9 -76.396243  157.351948  39.734369
4   44664       8  49.559341   86.044948   1.000000
5   11342       5  36.063459   71.383547  29.348293
6   59135       5   7.960739   -6.260968   4.092837
7   38911   

   GiftId  TripId   Latitude   Longitude     Weight
0   27902       3  11.582659  -84.482748  12.927131
1   24816       3  69.513318  108.904265   5.194177
2   44797       4  81.696803  -16.911040   8.996455
3    4971       6  22.254424    5.638346   1.000000
4   90746       4 -86.723247 -178.938258   4.243859
5   29463       9  -8.062325  -43.727325  27.207718
6   94611       8  -3.111132  -72.698829   5.225300
7   58010       0  40.404809   80.588672  17.422602
8    5384       1  26.186508   95.541109   1.000000
9   59825       9 -83.612751 -161.416018   9.276309
   GiftId  TripId   Latitude   Longitude     Weight
0   27902       1  11.582659  -84.482748  12.927131
1   24816       5  69.513318  108.904265   5.194177
2   44797       0  81.696803  -16.911040   8.996455
3    4971       6  22.254424    5.638346   1.000000
4   90746       0 -86.723247 -178.938258   4.243859
5   29463       8  -8.062325  -43.727325  27.207718
6   94611       8  -3.111132  -72.698829   5.225300
7   58010   

   GiftId  TripId   Latitude   Longitude     Weight
0   52098       5  67.410867 -137.807120   1.000000
1   34241       2 -73.333328   -2.074800  16.103185
2   31093       1  51.807544  -77.535795   3.438298
3   49028       5 -24.038943   26.997538   1.000000
4   76296       7  66.165395 -113.887911   1.000000
5   78797       5 -76.988194   65.359975  10.655647
6   81717       0  25.283137   25.245423   1.000000
7    7860       5  -1.048672   19.760419  33.071936
8   76985       9  42.423829   92.790339   1.000000
9   28146       5  18.700512  -10.528243   1.613729
Seed 8 7186 WRW: 3231611.774892069
   GiftId  TripId   Latitude   Longitude     Weight
0   90191       8 -85.042026   -4.888659   1.000000
1   34080       5 -74.107235  152.801711   1.000000
2   83310       6  28.735312   18.506059   1.000000
3   63220       3   1.630825  -67.350436  13.628196
4   59813       2   5.480585   19.771530  16.608510
5   76091       5  10.992409   77.150794   1.000000
6   28685       1 -83.796775 

   GiftId  TripId   Latitude   Longitude     Weight
0   75199       9  40.248613   -7.500744   1.000000
1   62741       6 -40.893074  -72.124816   1.000000
2   98745       9   6.436849   -7.760000  29.258394
3   60452       6  58.720323  122.751188  30.266995
4   50319       4  12.137589   98.892563   1.000000
5    6763       0 -15.823632   22.954256  50.000000
6   41066       0 -68.383073  -60.814475  18.852632
7   94271       4 -18.574434  126.409652   2.259105
8    8783       1 -15.711231   20.941727  38.686542
9   56933       5 -11.243392   28.815029  22.960253
   GiftId  TripId   Latitude   Longitude     Weight
0   75199       0  40.248613   -7.500744   1.000000
1   62741       9 -40.893074  -72.124816   1.000000
2   98745       4   6.436849   -7.760000  29.258394
3   60452       2  58.720323  122.751188  30.266995
4   50319       6  12.137589   98.892563   1.000000
5    6763       4 -15.823632   22.954256  50.000000
6   41066       4 -68.383073  -60.814475  18.852632
7   94271   

...

Seed 18 6705 WRW: 2721684.551933815
   GiftId  TripId   Latitude   Longitude     Weight
0   70491       4  37.805756  104.923506   1.000000
1   93058       3 -24.842539   17.971897   1.000000
2   25556       2  28.278913   -4.040024  24.466641
3    1018       9  64.603509 -133.606554  50.000000
4   33297       5 -79.943881 -135.489243  20.386566
5   88811       3  47.362309   92.176998   1.000000
6   19861       1 -70.853065   53.037410   1.000000
7    3836       9  60.519780   60.014804  45.694063
8    3796       5 -70.176102  135.712758  38.283102
9   30517       5  16.882828   -8.253457  17.704707
   GiftId  TripId   Latitude   Longitude     Weight
0   70491       0  37.805756  104.923506   1.000000
1   93058       1 -24.842539   17.971897   1.000000
2   25556       9  28.278913   -4.040024  24.466641
3    1018       3  64.603509 -133.606554  50.000000
4   33297       8 -79.943881 -135.489243  20.386566
5   88811       4  47.362309   92.176998   1.000000
6   19861       4 -70.853065

   GiftId  TripId   Latitude   Longitude     Weight
0   45805       5  60.996834 -164.866997  27.743327
1   85478       3  26.504863   19.113608   1.000000
2   83906       2  72.280762   84.880339  14.926895
3   56251       3  21.027854   79.683941  11.738434
4   67455       3  15.801904   45.316723   4.191018
5   13636       7  79.627232  -29.834291  27.531938
6   22671       2  48.484361  -75.056978   1.000000
7   87283       0  37.833668   95.452033   9.885730
8   41752       4  36.868297   69.125483  50.000000
9   92336       0 -82.076256  110.733839   1.000000
   GiftId  TripId   Latitude   Longitude     Weight
0   45805       9  60.996834 -164.866997  27.743327
1   85478       2  26.504863   19.113608   1.000000
2   83906       7  72.280762   84.880339  14.926895
3   56251       4  21.027854   79.683941  11.738434
4   67455       0  15.801904   45.316723   4.191018
5   13636       7  79.627232  -29.834291  27.531938
6   22671       3  48.484361  -75.056978   1.000000
7   87283   

   GiftId  TripId   Latitude   Longitude     Weight
0     211       3  70.551325   93.808767   1.791221
1   49493       2  28.749108   55.086731  13.914148
2   15849       4 -21.947557  -56.806367   1.000000
3   27806       4  66.861265 -122.342195  11.904212
4   11911       2  36.258045   40.445670  11.251445
5   43973       3 -88.342019  -75.914108   1.000000
6   87318       4  20.804066   53.734491  11.419579
7   11457       7 -41.494339  -67.207761  15.804222
8   75684       2  58.299769   37.188559   2.452166
9   97720       9  10.910921   43.254149   4.021719
   GiftId  TripId   Latitude   Longitude     Weight
0     211       2  70.551325   93.808767   1.791221
1   49493       9  28.749108   55.086731  13.914148
2   15849       4 -21.947557  -56.806367   1.000000
3   27806       0  66.861265 -122.342195  11.904212
4   11911       1  36.258045   40.445670  11.251445
5   43973       0 -88.342019  -75.914108   1.000000
6   87318       9  20.804066   53.734491  11.419579
7   11457   

   GiftId  TripId   Latitude   Longitude     Weight
0   29130       6  49.586381   19.932605  46.750401
1   88940       2  -8.352424  -37.392497   5.787946
2   80022       9  57.484664   14.743320  13.174278
3   50786       4  51.543286   49.840152  22.178868
4    1016       6 -77.663031  146.164707  22.578988
5   14515       3  -0.988552  -63.668640   1.000000
6   98513       1 -22.071812  122.924582  14.678464
7   58416       5  35.387927    1.650875  15.179186
8    3608       9   7.076671   43.938172  26.406423
9   71642       8  39.133275   -0.775633  34.927379
   GiftId  TripId   Latitude   Longitude     Weight
0   29130       5  49.586381   19.932605  46.750401
1   88940       5  -8.352424  -37.392497   5.787946
2   80022       5  57.484664   14.743320  13.174278
3   50786       6  51.543286   49.840152  22.178868
4    1016       1 -77.663031  146.164707  22.578988
5   14515       1  -0.988552  -63.668640   1.000000
6   98513       4 -22.071812  122.924582  14.678464
7   58416   

   GiftId  TripId   Latitude   Longitude     Weight
0   55543       8 -21.295952   26.835362   1.000000
1   48665       9 -80.035277  -76.885832   1.000000
2     545       8 -28.982095  122.933741   1.000000
3   16275       2  67.921639  -95.194002   5.858771
4    7436       3 -24.440761  -68.047273  20.187816
5   59752       9  27.494016   83.606718   1.000000
6   92467       0  47.023664  -93.360487  31.227980
7   79142       8   6.334103   27.294523  26.389314
8   95012       4  18.208238  -98.979208   7.794556
9   32924       3  65.265157   92.347309   1.000000
   GiftId  TripId   Latitude   Longitude     Weight
0   55543       2 -21.295952   26.835362   1.000000
1   48665       7 -80.035277  -76.885832   1.000000
2     545       5 -28.982095  122.933741   1.000000
3   16275       2  67.921639  -95.194002   5.858771
4    7436       7 -24.440761  -68.047273  20.187816
5   59752       3  27.494016   83.606718   1.000000
6   92467       9  47.023664  -93.360487  31.227980
7   79142   

In [19]:
RS_stat

NameError: name 'RS_stat' is not defined